In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [7]:
from abc import ABC, abstractmethod
from typing import List, Tuple, Any, Dict
import numpy as np

from rank_bm25 import BM25Okapi
from kiwipiepy import Kiwi


class Indexer(ABC):
    @abstractmethod
    def search(self, query_text: Any, top_k: int):
        pass


class KoreanBM25Indexer(Indexer):
    def __init__(self, corpus: List[Dict[str, Any]], top_k: int = 5):
        """
        corpus format:
        [
          {
            "id": chunk_id,
            "chunked_text": "...",
            "original_docid": ...
          },
          ...
        ]
        """
        self.top_k = top_k

        # 🔹 chunk id와 텍스트 분리
        self.doc_ids = [item["id"] for item in corpus]
        self.texts = [item["chunked_text"] for item in corpus]

        # 한국어 형태소 토크나이저
        self.kiwi = Kiwi()

        # 문서 토크나이즈
        tokenized_corpus = [self._tokenize(text) for text in self.texts]

        # BM25 인덱스 생성
        self.bm25 = BM25Okapi(tokenized_corpus)

    def _tokenize(self, text: str) -> List[str]:
        # 형태소 단위 토큰
        return [tok.form for tok in self.kiwi.tokenize(text)]

    def search(self, query_text: Any, top_k: int = None):
        if top_k is None:
            top_k = self.top_k

        if isinstance(query_text, str):
            q_tokens = self._tokenize(query_text)
            scores = np.asarray(self.bm25.get_scores(q_tokens))

            top_idx = np.argsort(scores)[::-1][:top_k]
            doc_ids = [self.doc_ids[i] for i in top_idx]
            top_scores = scores[top_idx]

            return doc_ids, top_scores

        elif isinstance(query_text, list):
            return [self.search(q, top_k) for q in query_text]

        else:
            raise NotImplementedError("Input type not str nor list.")


In [8]:
from pipeline.dataset.news import get_news_dataset

dataset = get_news_dataset()

In [18]:
from pipeline.dataset.newsqa import load_news_qa_dataset
newsqa = load_news_qa_dataset()

Loading NewsQA dataset from /workspace/final_project/qa/newsqa.json


In [9]:
from pipeline.retriever.embed import chunk_korean_sentence

In [10]:
points = []

for doc_idx, _ in enumerate(dataset['text']):
    
    chunk_texts = chunk_korean_sentence(dataset[doc_idx]['text'], chunk_size=1000, chunk_overlap=200)
    
    for chunk_idx, chunk_text in enumerate(chunk_texts):
        point = {
            "id": doc_idx*1000+chunk_idx,
            'chunked_text' : chunk_text,
            'original_docid' : doc_idx
        }
        points.append(point)

In [ ]:
indexer = KoreanBM25Indexer(corpus=points, top_k=100)

In [25]:
topk=5

In [21]:
results = []
from tqdm import tqdm
for q in tqdm([d['question'] for d in newsqa]):
    results.append(indexer.search(q))

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [06:20<00:00,  2.63it/s]


In [35]:
indexer.texts[indexer.doc_ids.index(1)]

'올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, 정부가 하반기에 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 결정한 가운데, 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했다.'

In [40]:
all_retrieved_docs = []
for docids, scores in results:
    topk_docs = docids[:topk]
    topk_scores = scores[:topk].tolist()
    for docid, score in zip(topk_docs, topk_scores):
        retrieved_doc = {
            'id': docid,
            'score': score,
            'chunked_text': indexer.texts[indexer.doc_ids.index(docid)],
            'original_chunk_id': docid % 1000,
            'original_docid': docid // 1000
        }
        all_retrieved_docs.append(retrieved_doc)

In [41]:
all_retrieved_docs

[{'id': 6039002,
  'score': 76.44372064741206,
  'chunked_text': '양자채널이용료는 속도 구분 없이 거리에 따라 3년 계약 기준 월 1억 3억 4000만원의 요금제로 구성된다. KT에 따르면 7년 이상 장기 계약 시 이용료는 최소 4000만원으로 3년 계약으로 이용할 때보다 50% 이상 저렴하다. 장비 임대료는 최대 1Gbps 속도용 장비는 월 1650만원이며 10Gbps용은 1760만원이다. 예컨대 서울과 용산 간 구간의 1Gbps 속도의 양자암호통신 서비스 이용료는 1억 1650만원 3년 계약 기준 이다. 서울과 부산 구간은 3억 8650만원의 월 요금으로 이용할 수 있다. KT는 양자암호통신 기술 선도를 위해 고속 양자암호통신 시스템 개발과 서울 부산 최장거리 양자암호통신 서비스 실증 국제표준화 선도 등 연구개발을 활발히 진행 중이다. 백승택 KT 데이터 인프라 디지털전환 DX 사업담당은 “KT는 세계에서 가장 많은 양자암호통신 관련 국제 표준을 바탕으로 공공 국방 금융 등 분야에 강력한 보안 서비스를 제공할 수 있다”며 “지속적인 서비스 개발 통해 대한민국은 물론 세계 양자암호통신 분야를 선도할 것”이라고 말했다.',
  'original_chunk_id': 2,
  'original_docid': 6039},
 {'id': 2818002,
  'score': 74.9387679751097,
  'chunked_text': '7년 이상 장기 계약 시 이용료는 최소 4000만원으로 3년 계약으로 이용할 때보다 50% 이상 저렴한 비용으로 안정적인 양자암호통신 서비스를 이용할 수 있다. 장비 임대료는 최대 1Gbps 속도용 장비는 월 1650만원이며 10Gbps용은 1760만원이다. 예컨대 서울과 용산 간 구간의 1Gbps 속도의 양자암호통신 서비스 이용료는 1억1650만원 3년 계약 기준 이며 서울과 부산 구간은 3억8650만원의 월 요금으로 이용할 수 있다. KT는 대한민국의 양자암호통신 기술 선도를 위

In [3]:
from pipeline.chain.generation import RAGChain
from pipeline.common import setup_logger

/venv/final_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
logger = setup_logger("ipynbDEMO")

In [5]:
rag = RAGChain(
    retrieval_type='hybrid',
    hybrid_alpha=0.5,
    encoder_name='bge',
    chunk_size=1000,
    overlap_size=200,
    generator_name='midm',
    top_k=5,
    generator_type='vllm',
    with_retrieval_results=True,
    logger=logger
)

[2025-12-19 19:35:57] INFO SentenceTransformer.py:219: Use pytorch device_name: cuda:0
[2025-12-19 19:35:57] INFO SentenceTransformer.py:227: Load pretrained SentenceTransformer: dragonkue/bge-m3-ko


Model is on device: cuda:0


In [13]:
rag.top_k

5

In [17]:
rag.ask(["집 근처에서 당근마켓 하는 법."])

Batches: 100%|██████████| 1/1 [00:00<00:00, 29.37it/s]


[('{"answer": "집 근처에서 당근마켓을 하는 법은 당근마켓 내 \'내 근처\' 탭의 \'당근미니\'에서 남의집 서비스를 이용하는 것입니다. 별도의 앱 설치 없이 서비스 페이지에서 바로 원하는 모임에 신청해 참여할 수 있으며, 모임은 동네 가게나 작업실 공방 가정집 등 호스트가 초청한 다양한 공간에서 열립니다."}',
  [{'id': 10101002,
    'score': 0.615136,
    'original_chunk_id': 2,
    'category': 'economy',
    'press': '서울신문 ',
    'link': 'https://n.news.naver.com/mnews/article/081/0003284684?sid=101',
    'bucket': 'train',
    'chunked_text': '정기 모임보다는 가볍게 ‘번개’ 갑작스러운 만남 를 할 수 있는 서비스로 실제로 최근 ‘같이해요’를 통해 동네 사람들과 함께 운동을 하거나 혼자 먹기는 힘든 삼겹살집을 같이 가는 등의 즉흥 모임이 빈번해지고 있습니다. 당근마켓은 앞으로도 단순한 중고거래 플랫폼에 머물지 않고 ‘하이퍼로컬’ 지역 밀착 생태계 조성을 이루겠다는 목표를 내걸고 있습니다. 당근마켓 관계자는 “사람과 사람 사이가 점점 멀어지는 언택트 시대에 지역 공동체와 이웃의 정을 느끼게 해 주는 서비스를 지향하는 것이 당근마켓의 DNA”라고 설명했습니다.',
    'original_docid': 10101,
    'dense_score_scaled': 1.0,
    'sparse_score_scaled': 0.0,
    'final_score': 0.5,
    'in_dense_topk': True,
    'in_sparse_topk': False},
   {'id': 11401001,
    'score': 55.77955353620989,
    'chunked_text': '지난해 당근마켓 10억 원 투자 후 4월 서울 연동 가입자 수 

In [59]:
answer1 = rag.ask(["서울 근교 공원을 알려줘"])
answer2 = rag.ask(["부산 근교 공원을 알려줘"])

[{'id': 26593001, 'score': 27.08803963993782, 'chunked_text': '애플스토어 가로수길점 초등생 아이패드 체험 행사 지구 보호 관련 만화 제작… 잠재고객층 록인 전략 신사 은행나무 공원에서 아이패드 만화에 들어갈 사진을 찍는 모습. 2022.07.01. 오현주 기자 © 뉴스1 서울 뉴스1 오현주 기자 제 또래 엄마들은 10명 중 2명만 아이폰 같은 애플 제품을 쓰던데 아이들은 안 그래요. 알파 세대 2010년 2024년생 는 키즈폰을 쓰고 있어도 아이폰을 쓰고 싶어 하죠. 지난 1일 오후 5시 서울 강남구 신사동 가로수길 인근 은행나무 공원에서 만난 학부모 A씨 44세 의 말이다. A씨는 이날 초등학교 3학년·5학년 아들과 애플스토어 가로수길점과 근처 놀이터에서 열린 애플 캠프 에 참가했다. 일찍부터 스마트폰 같은 전자기기에 친숙한 만8 12세 어린이들이 아이패드로 지구 보호 관련 만화를 만들어보는 프로그램이다. A씨의 자녀들은 또 다른 초등생 3명과 함께 아이패드 프로 11인치 3세대 를 들고 공원 속 나무와 비둘기 가족들의 사진을 찍고 이미지 위에 애플펜슬로 그림을 그렸다. 어린이들에게 아이패드로 만화 그리는 법을 알려주는 애플스토어 직원2022.07.01. 오현주 기자© 뉴스1 ◇애플 3년 만에 어린이 대상 애플 캠프 …아이패드 만화책 제작 행사 이날 총 8명 학생 5명·부모 3명 이 참여한 행사는 애플이 3년 만에 연 오프라인 어린이 캠프의 일환인 아트 연구소 가족과 함께 만화책 만들기 대모험 이다. 8월 31일까지 국내 3곳 가로수길·여의도·명동점 을 포함한 전 세계 애플스토어에서 매주 열린다. 어릴 때부터 휴대전화에 익숙한 알파 세대 에게 애플 기기를 빌려줘 브랜드를 알리고 어린이를 미래 주요 구매층으로 포섭하기 위한 일종의 록인 Lock in 전략으로 분석된다. 이날 가로수길점 행사는 다섯 가지 활동으로 구성됐다. Δ아이패드 작동법 교육 등 워밍업 10분 Δ만화책 만들기 시범 20분 Δ공원 주변 둘러보기

In [62]:
answer1[0][1]

[{'id': 26593001,
  'score': 27.08803963993782,
  'chunked_text': '애플스토어 가로수길점 초등생 아이패드 체험 행사 지구 보호 관련 만화 제작… 잠재고객층 록인 전략 신사 은행나무 공원에서 아이패드 만화에 들어갈 사진을 찍는 모습. 2022.07.01. 오현주 기자 © 뉴스1 서울 뉴스1 오현주 기자 제 또래 엄마들은 10명 중 2명만 아이폰 같은 애플 제품을 쓰던데 아이들은 안 그래요. 알파 세대 2010년 2024년생 는 키즈폰을 쓰고 있어도 아이폰을 쓰고 싶어 하죠. 지난 1일 오후 5시 서울 강남구 신사동 가로수길 인근 은행나무 공원에서 만난 학부모 A씨 44세 의 말이다. A씨는 이날 초등학교 3학년·5학년 아들과 애플스토어 가로수길점과 근처 놀이터에서 열린 애플 캠프 에 참가했다. 일찍부터 스마트폰 같은 전자기기에 친숙한 만8 12세 어린이들이 아이패드로 지구 보호 관련 만화를 만들어보는 프로그램이다. A씨의 자녀들은 또 다른 초등생 3명과 함께 아이패드 프로 11인치 3세대 를 들고 공원 속 나무와 비둘기 가족들의 사진을 찍고 이미지 위에 애플펜슬로 그림을 그렸다. 어린이들에게 아이패드로 만화 그리는 법을 알려주는 애플스토어 직원2022.07.01. 오현주 기자© 뉴스1 ◇애플 3년 만에 어린이 대상 애플 캠프 …아이패드 만화책 제작 행사 이날 총 8명 학생 5명·부모 3명 이 참여한 행사는 애플이 3년 만에 연 오프라인 어린이 캠프의 일환인 아트 연구소 가족과 함께 만화책 만들기 대모험 이다. 8월 31일까지 국내 3곳 가로수길·여의도·명동점 을 포함한 전 세계 애플스토어에서 매주 열린다. 어릴 때부터 휴대전화에 익숙한 알파 세대 에게 애플 기기를 빌려줘 브랜드를 알리고 어린이를 미래 주요 구매층으로 포섭하기 위한 일종의 록인 Lock in 전략으로 분석된다. 이날 가로수길점 행사는 다섯 가지 활동으로 구성됐다. Δ아이패드 작동법 교육 등 워밍업 10분 Δ만화책 만들기 시범 20분 Δ공원 주변 

In [63]:
answer2[0][1]

[{'id': 2696002,
  'score': 30.794056415436927,
  'chunked_text': '현지 부동산 전문가는 촉진 구역 지정 15년 만에 부산 시민공원 재정비촉진지구의 윤곽이 드러날 것 이라며 사업이 완공되면 부산 중심지에서 향후 총 8000여가구가 넘는 대단지 주거타운을 형성해 새로운 부산 랜드마크 지역으로 변화할 것 이라고 했다. 부산 시민공원 재정비촉진지구의 개발 면적은 43만6378㎡로 47만여㎡ 규모 부산 도심권 허파 기능을 담당하는 부산 시민공원을 둘러싼 채 5개 구역으로 나누어져 있다. 2007년 정부의 도심 내 뉴타운 조성 추진 정책에 힘입어 재정비촉진지구로 지정돼 공동주택과 각종 주거 인프라가 구축될 예정이다.',
  'original_chunk_id': 2,
  'original_docid': 2696},
 {'id': 1496000,
  'score': 29.547269717618192,
  'chunked_text': '한국마사회 3년만에 야간 경마…78월 금·토요일 시행\n한국마사회는 고객들에게 다양한 볼거리를 제공하고 무더운 여름 경주마와 기수 등을 보호하기 위해 7월 1일부터 약 두 달간 야간경마를 개최한다고 밝혔다. 사진은 서울경마공원의 야간 경마 모습. 한국마사회 7월 1일부터 9주 간 매주 금·토요일 야간경마가 열린다. 한국마사회는 고객들에게 다양한 볼거리를 제공하고 무더운 여름 경주마와 기수 등을 보호하기 위해 1일부터 약 두 달간 야간경마를 개최한다고 밝혔다. 혹서기 기간 밤에 진행되는 야간경마는 약 3년 만에 재개된다. 올해 야간경마는 7월 1일부터 8월 28일까지 9주 간 매주 금요일과 토요일 시행된다. 금요일에는 부산경남과 제주 경마공원에서 야간 경주가 열리며 토요일에는 서울에서 열린다. 단 토요일 부산경남 경마공원은 서울경마공원 휴장 기간인 8월 첫째주에도 야간 경마를 진행한다. 이에 따라 금요일과 토요일 마지막 경주 출발시각은 밤 9시다. 일요일은 기존처럼 서울·부산경남 경주가 주간 

In [67]:
from copy import deepcopy

def minmax_map(results, score_key="score"):
    """results(list[dict]) -> {id: scaled_score}"""
    if not results:
        return {}

    scores = [r[score_key] for r in results]
    mn, mx = min(scores), max(scores)

    if mx == mn:
        # 전부 같은 점수면 모두 1.0(또는 0.0)로 둬도 되지만,
        # topK 내 상대순위가 의미 없으므로 1.0으로 통일하는 편이 보통 무난
        return {r["id"]: 1.0 for r in results}

    return {r["id"]: (r[score_key] - mn) / (mx - mn) for r in results}
def hybrid_union_merge(dense_results, sparse_results, alpha: float):
    """
    dense_results, sparse_results: topK 결과(list[dict]) - id 집합이 달라도 됨
    alpha: dense 가중치 (0~1)
    return: union된 문서들을 final_score로 내림차순 정렬한 list[dict]
    """
    assert 0.0 <= alpha <= 1.0

    dense_scaled = minmax_map(dense_results)
    sparse_scaled = minmax_map(sparse_results)

    dense_map = {r["id"]: r for r in dense_results}
    sparse_map = {r["id"]: r for r in sparse_results}

    all_ids = set(dense_map) | set(sparse_map)

    merged = []
    for doc_id in all_ids:
        d_item = dense_map.get(doc_id)
        s_item = sparse_map.get(doc_id)

        d_score = dense_scaled.get(doc_id, 0.0)   # 없으면 0
        s_score = sparse_scaled.get(doc_id, 0.0)  # 없으면 0

        final = alpha * d_score + (1 - alpha) * s_score

        # base item: dense에 있으면 dense 메타를, 없으면 sparse 메타를 사용
        base = deepcopy(d_item if d_item is not None else s_item)

        base["dense_score_scaled"] = d_score
        base["sparse_score_scaled"] = s_score
        base["final_score"] = final
        base["in_dense_topk"] = d_item is not None
        base["in_sparse_topk"] = s_item is not None

        merged.append(base)

    merged.sort(key=lambda x: x["final_score"], reverse=True)
    return merged

In [69]:
answer3 = [{'id': 26593001,
  'score': 27.08803963993782,
  'chunked_text': '애플스토어 가로수길점 초등생 아이패드 체험 행사 지구 보호 관련 만화 제작… 잠재고객층 록인 전략 신사 은행나무 공원에서 아이패드 만화에 들어갈 사진을 찍는 모습. 2022.07.01. 오현주 기자 © 뉴스1 서울 뉴스1 오현주 기자 제 또래 엄마들은 10명 중 2명만 아이폰 같은 애플 제품을 쓰던데 아이들은 안 그래요. 알파 세대 2010년 2024년생 는 키즈폰을 쓰고 있어도 아이폰을 쓰고 싶어 하죠. 지난 1일 오후 5시 서울 강남구 신사동 가로수길 인근 은행나무 공원에서 만난 학부모 A씨 44세 의 말이다. A씨는 이날 초등학교 3학년·5학년 아들과 애플스토어 가로수길점과 근처 놀이터에서 열린 애플 캠프 에 참가했다. 일찍부터 스마트폰 같은 전자기기에 친숙한 만8 12세 어린이들이 아이패드로 지구 보호 관련 만화를 만들어보는 프로그램이다. A씨의 자녀들은 또 다른 초등생 3명과 함께 아이패드 프로 11인치 3세대 를 들고 공원 속 나무와 비둘기 가족들의 사진을 찍고 이미지 위에 애플펜슬로 그림을 그렸다. 어린이들에게 아이패드로 만화 그리는 법을 알려주는 애플스토어 직원2022.07.01. 오현주 기자© 뉴스1 ◇애플 3년 만에 어린이 대상 애플 캠프 …아이패드 만화책 제작 행사 이날 총 8명 학생 5명·부모 3명 이 참여한 행사는 애플이 3년 만에 연 오프라인 어린이 캠프의 일환인 아트 연구소 가족과 함께 만화책 만들기 대모험 이다. 8월 31일까지 국내 3곳 가로수길·여의도·명동점 을 포함한 전 세계 애플스토어에서 매주 열린다. 어릴 때부터 휴대전화에 익숙한 알파 세대 에게 애플 기기를 빌려줘 브랜드를 알리고 어린이를 미래 주요 구매층으로 포섭하기 위한 일종의 록인 Lock in 전략으로 분석된다. 이날 가로수길점 행사는 다섯 가지 활동으로 구성됐다. Δ아이패드 작동법 교육 등 워밍업 10분 Δ만화책 만들기 시범 20분 Δ공원 주변 둘러보기·사진 촬영 Δ만화책 디자인 30분 Δ발표 25분 다.',
  'original_chunk_id': 1,
  'original_docid': 26593},
 {'id': 27120004,
  'score': 27.06476880084895,
  'chunked_text': '하늘공원 코스는 1.2km로 연인끼리 데이트 즐기거나 혼자 고요히 벤치에 앉아 독서를 즐기기 좋다. 유아숲체험원 코스는 모래 놀이터와 트리 하우스가 있어 영유아를 동반한 가족들이 걷기 좋은 숲길이다. ◇ 이국적인 분위기 ‘서서울호수공원 몬드리안 정원’ 서서울호수공원에는 비행기가 지나면 분수가 솟는 소리 분수가 있다. 생각보다 가까이에서 날아가는 비행기와 분수를 바라보면 마치 어디론가 여행을 떠나는 기분을 느낄 수 있다. 호수 주위를 한 바퀴 걷고 나서 가만히 데크에 앉아 분수를 바라보는 것도 좋지만 이국적인 느낌의 몬드리안 정원도 놓치지 말고 꼭 만나보자. 과거 정수장의 흔적인 침전조의 콘크리트벽과 덩굴식물 다양한 꽃들이 어우러진 정원은 곳곳에 다양한 벤치와 햇빛을 막아주는 그늘 공간이 있어 어디서든 휴식할 수 있다. 아름다운 정원에서 인생사진도 남기며 여행 온 듯한 힐링의 시간을 가져보자. 서서울호수공원 사진 서울시 ◇ 비가 와도 괜찮은 ‘책쉼터 공원 여행’ 공원이 야외공간이라고만 생각했다면 공원속 책쉼터에 방문해보자. 시원한 공간에서 책을 읽으며 창 밖 자연을 즐길 수 있어 무더운 날에도 비가 오는 날에도 공원을 즐기기 좋다. 특히 자연 훼손을 최소화하고 건축적으로도 아름답게 지어 감성적인 북카페에 온 듯한 기분을 느낄 수 있다. 공원 내 책쉼터는 응봉근린공원 양천공원 쌍문근린공원 구로구 천왕산 근린공원에서 만나볼 수 있다.',
  'original_chunk_id': 4,
  'original_docid': 27120},
 {'id': 20561002,
  'score': 26.550626476328944,
  'chunked_text': '김해공원묘원도 성묘객들을 설득해 묘 1만7000여기에 있는 가짜꽃을 3월에 모두 수거했다. 최이천 김해공원묘원 이사장은 “김해시와 함께 민원인을 설득하고 성묘객에게 근절 취지를 잘 설명해 가짜꽃을 모두 제거할 수 있었다”며 “환경에 유해한 가짜꽃이 사라져 공원묘원이 깔끔해지고 보기 좋다는 반응이 많다”고 귀띔했다. 민원 때문에 어려움 겪기도 묘 5500기가 있는 영락공원묘원은 5월 가짜꽃 수거를 마쳤지만 그 과정에서 민원으로 곤혹스러웠다고 한다. 김대석 영락공원묘원 대표는 “왜 사유재산인 가짜꽃을 함부로 수거하냐며 민원인이 경찰서에 신고했다”며 “대전현충원이나 서울현충원처럼 국가에서 관리하는 곳부터 가짜꽃을 먼저 없앴으면 더 나았겠다”고 말했다. 묘 8500기를 관리하는 김해하늘공원은 민원이 많이 들어와 가짜꽃 수거가 중단된 상태다. 김해하늘공원 관계자는 “생화는 금방 시든다는 점을 지적하며 민원이 많이 들어와서 현재는 가짜꽃을 수거할 수 없는 상황”이라고 토로했다. 윤수근 김해하늘공원 이사장은 “이용객들에게 가짜꽃의 유해성을 더 알리고 반입 금지를 홍보해서 올 추석 전후로 수거를 본격 시작하겠다”고 설명했다. 가짜꽃 대체품 마련 나서 김해시는 가짜꽃 사용 금지에 따른 시민들 불만을 해결하기 위해 대체품 마련에 힘쓰고 있다. 가짜꽃 근절 사업을 담당하고 있는 강문숙 청소행정과 청소관리팀장은 “자조금협의회·영남화훼원예농협과 협력해 생화보다 오래 유지되는 드라이플라워를 저렴한 값으로 시민들에게 제공할 예정”이라며 “시민들이 헌화를 구입할 때 불편함을 느끼지 않도록 계속 지원할 방침”이라고 밝혔다. 강 팀장은 이어 “드라이플라워 자판기를 8월까지 4개 공원묘원에 설치해서 올 추석부터 시민들이 손쉽게 구매할 수 있도록 하겠다”고 강조했다. 영남화훼원예농협도 드라이플라워 공급 꽃 냉장고 기증 등을 통해 생화 소비 촉진에 힘쓸 예정이다.',
  'original_chunk_id': 2,
  'original_docid': 20561},
 {'id': 1496000,
  'score': 26.377055290675302,
  'chunked_text': '한국마사회 3년만에 야간 경마…78월 금·토요일 시행\n한국마사회는 고객들에게 다양한 볼거리를 제공하고 무더운 여름 경주마와 기수 등을 보호하기 위해 7월 1일부터 약 두 달간 야간경마를 개최한다고 밝혔다. 사진은 서울경마공원의 야간 경마 모습. 한국마사회 7월 1일부터 9주 간 매주 금·토요일 야간경마가 열린다. 한국마사회는 고객들에게 다양한 볼거리를 제공하고 무더운 여름 경주마와 기수 등을 보호하기 위해 1일부터 약 두 달간 야간경마를 개최한다고 밝혔다. 혹서기 기간 밤에 진행되는 야간경마는 약 3년 만에 재개된다. 올해 야간경마는 7월 1일부터 8월 28일까지 9주 간 매주 금요일과 토요일 시행된다. 금요일에는 부산경남과 제주 경마공원에서 야간 경주가 열리며 토요일에는 서울에서 열린다. 단 토요일 부산경남 경마공원은 서울경마공원 휴장 기간인 8월 첫째주에도 야간 경마를 진행한다. 이에 따라 금요일과 토요일 마지막 경주 출발시각은 밤 9시다. 일요일은 기존처럼 서울·부산경남 경주가 주간 경마로 시행된다. 고객 입장시간도 변동된다. 금요일과 토요일은 낮 12시 30분부터 입장이 시작되며 일요일은 오전 9시 30분이다. 첫 경주 출발 시각도 금요일 오후 2시 10분 토요일은 오후 2시로 변경된다. 한국마사회는 여름철 혹서기에 경마장별로 휴장 기간을 운영한다. 제주 경마공원은 7월 마지막 주에 휴장이 예정돼 있으며 그 다음 주인 8월 첫째 주는 서울 경마공원 8월 둘째 주는 부산경남 경마공원이 돌아가면서 휴장을 진행한다.\n한국들에게 다양한 볼거리를 제공하고 무더운 여름 경주마와 기수 등을 보호하기 위해 한국마사회는 7월 1일부터 8월 28일까지 9주 간 매주 금·토요일 야간경마를 개최하며 금요일에는 부산경남과 제주 경마공원에서 야간 경주가 열리며 토요일에는 서울에서 열리고 금요일과 토요일 마지막 경주 출발시각은 밤 9시라고 밝혔다.',
  'original_chunk_id': 0,
  'original_docid': 1496},
 {'id': 27120003,
  'score': 26.21943343662731,
  'chunked_text': '특히 데크산책로 양쪽으로 40여 종의 키 작은 나무와 풀꽃이 심어져 있어 포토존으로도 손색이 없다. 무더운 올여름 보라매공원에서 책 한 권을 읽으며 몸과 마음 모두 충전해보자. ◇ 돗자리만 있다면 ‘시민의숲공원’ 어디서나 피크닉 한적하고 울창한 숲속에서 피크닉하는 기분을 느껴보고 싶다면 우리나라 최초 숲 개념을 도입한 공원인 양재 시민의숲을 추천한다. 30년 이상이 된 나무들이 도심 내 울창한 숲을 이루고 있어 시민들이 초록의 싱그러움화 함께 시원한 나무 그늘을 쉽게 찾을 수 있다. 공원이 넓고 테이블이 놓여진 곳과 돗자리를 펼만한 장소가 많아 붐비지 않고 여유있게 공원 속 힐링을 즐길 수 있다. 또한 서울둘레길 4코스와 연결되는 공원으로 둘레길을 걷는 도중 잠시 멈춰 간식시간을 갖기에도 좋은 곳이다. 시민의숲 숲속 햇살 사진 서울시 ◇ 돗자리 펴기 딱 좋은 ‘난지천공원 잔디광장’ 난지잔디광장이 자리한 난지천공원은 생태환경이 복원되고 있는 곳으로 다양한 수생식물이 서식하고 물고기와 새가 찾아드는 공간이다. 특히 탁 트인 잔디광장은 돗자리를 펴고 피크닉을 즐기기 좋은 장소이다. 너른 잔디 위에서 가족 친구끼리 배드민턴 등 여가활동을 즐기기에도 손색이 없다. 잔디광장 양옆으로는 각각 어린이 놀이터와 오리들을 만날 수 있는 오리 연못이 있어 영유아를 동반한 가족들이 찾기에도 좋은 장소다. ◇ 숲길을 걷는 기분 ‘월드컵공원 메타세콰이아길’ 울창한 나무아래를 걸으며 생각을 정리하고 싶을 때가 있다. 그럴 때 먼 곳까지 가지 않아도 월드컵공원에서 메타세쿼이아길을 걸어보자. 하늘공원 코스와 유아숲체험원 코스 총 2갈래 길을 만날 수 있다. 하늘공원 코스는 1.2km로 연인끼리 데이트 즐기거나 혼자 고요히 벤치에 앉아 독서를 즐기기 좋다. 유아숲체험원 코스는 모래 놀이터와 트리 하우스가 있어 영유아를 동반한 가족들이 걷기 좋은 숲길이다. ◇ 이국적인 분위기 ‘서서울호수공원 몬드리안 정원’ 서서울호수공원에는 비행기가 지나면 분수가 솟는 소리 분수가 있다.',
  'original_chunk_id': 3,
  'original_docid': 27120},
]

In [70]:
hybrid_union_merge(answer3, answer2[0][1], alpha=0.5)

[{'id': 2696002,
  'score': 30.794056415436927,
  'chunked_text': '현지 부동산 전문가는 촉진 구역 지정 15년 만에 부산 시민공원 재정비촉진지구의 윤곽이 드러날 것 이라며 사업이 완공되면 부산 중심지에서 향후 총 8000여가구가 넘는 대단지 주거타운을 형성해 새로운 부산 랜드마크 지역으로 변화할 것 이라고 했다. 부산 시민공원 재정비촉진지구의 개발 면적은 43만6378㎡로 47만여㎡ 규모 부산 도심권 허파 기능을 담당하는 부산 시민공원을 둘러싼 채 5개 구역으로 나누어져 있다. 2007년 정부의 도심 내 뉴타운 조성 추진 정책에 힘입어 재정비촉진지구로 지정돼 공동주택과 각종 주거 인프라가 구축될 예정이다.',
  'original_chunk_id': 2,
  'original_docid': 2696,
  'dense_score_scaled': 0.0,
  'sparse_score_scaled': 1.0,
  'final_score': 0.5,
  'in_dense_topk': False,
  'in_sparse_topk': True},
 {'id': 26593001,
  'score': 27.08803963993782,
  'chunked_text': '애플스토어 가로수길점 초등생 아이패드 체험 행사 지구 보호 관련 만화 제작… 잠재고객층 록인 전략 신사 은행나무 공원에서 아이패드 만화에 들어갈 사진을 찍는 모습. 2022.07.01. 오현주 기자 © 뉴스1 서울 뉴스1 오현주 기자 제 또래 엄마들은 10명 중 2명만 아이폰 같은 애플 제품을 쓰던데 아이들은 안 그래요. 알파 세대 2010년 2024년생 는 키즈폰을 쓰고 있어도 아이폰을 쓰고 싶어 하죠. 지난 1일 오후 5시 서울 강남구 신사동 가로수길 인근 은행나무 공원에서 만난 학부모 A씨 44세 의 말이다. A씨는 이날 초등학교 3학년·5학년 아들과 애플스토어 가로수길점과 근처 놀이터에서 열린 애플 캠프 에 참가했다. 일찍부터 스마트폰 같은 전자기기에